# Interactive Dashboard
## Visualizations
- Effective & Expressive

- Have at least 4 distinct visual representations coded in Altair

- Custom-designed visualization not from an off-the-shelf library. Don’t copy the entire design from an existing visualization.

- Individual visualization components may be copied from elsewhere (with attribution). When you attribute, make sure you include a link to the original viz and how you changed it to suite your purposes

- Demonstrate appropriate and effective use of basic visual encodings: spatial layout, color, size, shape, etc.

- Provide a fair representation of the underlying data.

- The data / visualization should address an interesting question relevant to the partner or their users and convincingly solve target user tasks.

## Presentation
- Layout should use screen space wisely. Layouts that are too sparse force the user to scroll unnecessarily. Layouts that are too dense are cluttered.

- Dashboard should be in a grid (i.e., 2 rows, 2 columns if having 4)

- Use consistent color encodings when applicable across visual representations

- Interface should include enough headings, legends, axis labels, and any other explanatory text needed to make it comprehensible.

- Below each viz in the view there should be text that details what task the viz addresses.

## Interaction Elements
### Events
- The viz. in the dashboard must support

- support mouse-based interaction (e.g., hover, click, drag, tooltip)

- at least one UI widget (eg. dropdown, radio button, range slider).

- at least one advance direct interaction selection technique such as selection interval or selection point.

### Functionality
- at least one bi-directional linking across two vizzes (i.e., the change in one elicits a change in the other and vice versa)

- at least one use of filtering or drilling.

In [1]:
import pandas as pd
import altair as alt

In [6]:
df = pd.read_csv("http://data.insideairbnb.com/canada/bc/vancouver/2023-09-06/data/listings.csv")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6695 entries, 0 to 6694
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6695 non-null   int64  
 1   listing_url                                   6695 non-null   object 
 2   scrape_id                                     6695 non-null   int64  
 3   last_scraped                                  6695 non-null   object 
 4   source                                        6695 non-null   object 
 5   name                                          6695 non-null   object 
 6   description                                   6677 non-null   object 
 7   neighborhood_overview                         4433 non-null   object 
 8   picture_url                                   6695 non-null   object 
 9   host_id                                       6695 non-null   i

1. Tak: Tick for price distribution (select price range)


3. Tak: Boxplot (review_scores_rating, review_scores_accuracy, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value) rating for selected listings

4. Rachel pie for selected listings's roomtype

5. Chloe: Cloropeth map Summary of each neibourhood for selected area: # superhost, # listings, average availability ?? on a map

6. toggle for superhost ???

7. Rachel Listings(Count) vs. occupancy bar per selected neibourhood


2. dropdown to select a neighbourhood.

In [ ]:
vancouver_map = alt.topo_feature(data.world_110m.url, 'countries')